# Credit Data Visualization Demo: Classifier Model Interpreter

This notebook demonstrates all visualization capabilities using **credit/lending data**:
- **Target Variable**: `net_booking` (loan booking/conversion)
- **Features**: FICO score, LTV, DTI, Financing Amount

## Package Features Demonstrated:

1. **Interactive Plotly visualizations** instead of static matplotlib
2. **Prediction surface visualizations** (contour & 3D)
3. **Threshold detection** - finds non-linear breakpoints automatically
4. **Segment discovery** - finds behavioral groups traditional analytics misses
5. **Local explanation plots** - waterfall charts for individual predictions
6. **Simple, integrated API** - one Interpreter class for everything

## Setup

In [1]:
import sys
from pathlib import Path

parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from src.core import Interpreter
import warnings
warnings.filterwarnings('ignore')

print("Setup complete")

Setup complete


In [2]:
# Load the credit data
data_path = Path.cwd().parent / 'data' / 'test_credit_data.csv'
df = pd.read_csv(data_path)

print(f"Dataset: {df.shape[0]:,} rows, {df.shape[1]} columns")
print(f"Booking rate: {df['net_booking'].mean():.1%}")
print(f"\nFeatures: {list(df.columns)}")
print(f"\nData sample:")
df.head()

Dataset: 100,000 rows, 6 columns
Booking rate: 32.2%

Features: ['FICO', 'LTV', 'DTI', 'Fin_amt', 'application_date', 'net_booking']

Data sample:


,FICO,LTV,DTI,Fin_amt,application_date,net_booking
0,702,95.5,21.3,30634.0,2023-11-26 15:24:39.313421,0
1,633,70.5,33.9,7688.0,2023-11-26 15:24:39.313421,0
2,635,94.4,29.4,30203.0,2023-11-26 15:24:39.313421,0
3,718,94.3,43.9,63653.0,2023-11-26 15:24:39.313421,0
4,736,96.2,22.1,27488.0,2023-11-26 15:24:39.313421,0


In [3]:
# Prepare data - drop non-feature columns
X = df.drop(['application_date', 'net_booking'], axis=1).copy()
y = df['net_booking'].values

print("Features used for modeling:")
print(f"  - FICO: Credit score ({X['FICO'].min():.0f} - {X['FICO'].max():.0f})")
print(f"  - LTV: Loan-to-value ratio ({X['LTV'].min():.1f}% - {X['LTV'].max():.1f}%)")
print(f"  - DTI: Debt-to-income ratio ({X['DTI'].min():.1f}% - {X['DTI'].max():.1f}%)")
print(f"  - Fin_amt: Financing amount (${X['Fin_amt'].min():,.0f} - ${X['Fin_amt'].max():,.0f})")

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain size: {len(X_train):,}")
print(f"Test size: {len(X_test):,}")

Features used for modeling:
  - FICO: Credit score (447 - 842)
  - LTV: Loan-to-value ratio (50.0% - 130.0%)
  - DTI: Debt-to-income ratio (10.0% - 60.0%)
  - Fin_amt: Financing amount ($5,000 - $75,000)

Train size: 80,000
Test size: 20,000


In [4]:
# Train model
model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    eval_metric='logloss'
)
model.fit(X_train, y_train)

print(f"Test accuracy: {model.score(X_test, y_test):.3f}")

Test accuracy: 0.703


In [5]:
# Initialize Interpreter
interp = Interpreter(model, X_test, y_test, config='detailed_analysis')

print("Interpreter ready")

Interpreter ready


---
## 1. Global Feature Importance

**What it shows:** Which features matter most for predicting loan bookings

In [6]:
fig = interp.plot_global_importance(top_n=10)
fig.show()

---
## 2. Beeswarm Plot

**What it shows:** Distribution of SHAP values for each feature

- Each dot = one application
- Color = feature value (red=high, blue=low)
- X position = impact on booking probability

In [7]:
fig = interp.plot_beeswarm(top_n=4)
fig.show()

---
## 3. Feature Dependence Plots

**What it shows:** How each feature value affects booking probability

Key questions answered:
- At what FICO score does booking probability increase?
- How does LTV affect approval?
- Is there a DTI threshold?

In [8]:
# FICO Score dependence
fig = interp.plot_dependence('FICO')
fig.show()

print("\nInterpretation: Higher FICO scores should increase booking probability")


Interpretation: Higher FICO scores should increase booking probability


In [9]:
# LTV dependence
fig = interp.plot_dependence('LTV')
fig.show()

print("\nInterpretation: How does loan-to-value ratio affect booking?")


Interpretation: How does loan-to-value ratio affect booking?


In [10]:
# DTI dependence
fig = interp.plot_dependence('DTI')
fig.show()

print("\nInterpretation: Higher DTI typically reduces booking probability")


Interpretation: Higher DTI typically reduces booking probability


In [11]:
# Financing amount dependence
fig = interp.plot_dependence('Fin_amt')
fig.show()

print("\nInterpretation: How does loan size affect booking probability?")


Interpretation: How does loan size affect booking probability?


---
# Threshold Detection

## 4. Automatic Threshold Detection

**What it does:** Finds values where feature effects CHANGE significantly

**Why it matters for credit:** 
- Find the FICO score where approval rates jump
- Identify DTI cutoffs where risk increases
- Discover LTV thresholds

**NATIVE SHAP DOESN'T HAVE THIS!**

In [12]:
# Detect thresholds for FICO
threshold_result = interp.detect_thresholds('FICO')

print("="*70)
print("THRESHOLD DETECTION: FICO Score")
print("="*70)
print(f"\nMethod: {threshold_result['method']}")
print(f"Samples analyzed: {threshold_result['n_samples']:,}")

print("\nDetected Thresholds:")
print("-"*70)
for t in threshold_result['thresholds']:
    print(f"\n  Threshold at FICO: {t['value']:.0f}")
    print(f"    SHAP before: {t['shap_before']:.4f}")
    print(f"    SHAP after:  {t['shap_after']:.4f}")
    print(f"    Effect change: {t['effect_change']:+.4f}")
    print(f"    Confidence: {t['confidence']:.1%}")
    print(f"    -> {t['interpretation']}")

THRESHOLD DETECTION: FICO Score

Method: changepoint
Samples analyzed: 2,000

Detected Thresholds:
----------------------------------------------------------------------

  Threshold at FICO: 619
    SHAP before: -1.4235
    SHAP after:  0.1354
    Effect change: +1.5588
    Confidence: 100.0%
    -> Effect dramatically increases at FICO=619.00: from -1.423 to 0.135

  Threshold at FICO: 658
    SHAP before: -1.1395
    SHAP after:  0.3359
    Effect change: +1.4755
    Confidence: 100.0%
    -> Effect dramatically increases at FICO=658.00: from -1.140 to 0.336

  Threshold at FICO: 582
    SHAP before: -1.4387
    SHAP after:  0.0007
    Effect change: +1.4394
    Confidence: 100.0%
    -> Effect dramatically increases at FICO=582.00: from -1.439 to 0.001


In [13]:
# Detect thresholds for DTI
threshold_result = interp.detect_thresholds('DTI')

print("="*70)
print("THRESHOLD DETECTION: DTI (Debt-to-Income)")
print("="*70)

print("\nDetected Thresholds:")
for t in threshold_result['thresholds']:
    print(f"\n  Threshold at DTI: {t['value']:.1f}%")
    print(f"    Effect change: {t['effect_change']:+.4f}")
    print(f"    -> {t['interpretation']}")

THRESHOLD DETECTION: DTI (Debt-to-Income)

Detected Thresholds:

  Threshold at DTI: 50.8%
    Effect change: -0.5097
    -> Effect dramatically decreases at DTI=50.80: from -0.004 to -0.514

  Threshold at DTI: 45.9%
    Effect change: -0.4622
    -> Effect dramatically decreases at DTI=45.90: from 0.014 to -0.448

  Threshold at DTI: 41.0%
    Effect change: -0.3110
    -> Effect dramatically decreases at DTI=41.00: from 0.032 to -0.279


In [14]:
# Detect thresholds for LTV
threshold_result = interp.detect_thresholds('LTV')

print("="*70)
print("THRESHOLD DETECTION: LTV (Loan-to-Value)")
print("="*70)

print("\nDetected Thresholds:")
for t in threshold_result['thresholds']:
    print(f"\n  Threshold at LTV: {t['value']:.1f}%")
    print(f"    Effect change: {t['effect_change']:+.4f}")
    print(f"    -> {t['interpretation']}")

THRESHOLD DETECTION: LTV (Loan-to-Value)

Detected Thresholds:

  Threshold at LTV: 121.0%
    Effect change: -0.9120
    -> Effect dramatically decreases at LTV=121.00: from -0.001 to -0.913

  Threshold at LTV: 113.2%
    Effect change: -0.7363
    -> Effect dramatically decreases at LTV=113.20: from 0.051 to -0.685

  Threshold at LTV: 100.4%
    Effect change: -0.6640
    -> Effect dramatically decreases at LTV=100.40: from 0.176 to -0.488


In [15]:
# Detect thresholds across ALL features
all_thresholds = interp.detect_all_thresholds(top_n_features=4)

print("ALL DETECTED THRESHOLDS")
print("="*70)
all_thresholds[['feature', 'value', 'effect_change', 'interpretation']]

ALL DETECTED THRESHOLDS


,feature,value,effect_change,interpretation
0,FICO,619.0,1.558848,Effect dramatically increases at FICO=619.00: ...
1,FICO,658.0,1.475476,Effect dramatically increases at FICO=658.00: ...
2,FICO,582.0,1.439448,Effect dramatically increases at FICO=582.00: ...
3,LTV,121.0,-0.911983,Effect dramatically decreases at LTV=121.00: f...
4,LTV,113.2,-0.736252,Effect dramatically decreases at LTV=113.20: f...
5,LTV,100.4,-0.664027,Effect dramatically decreases at LTV=100.40: f...
6,DTI,50.8,-0.509693,Effect dramatically decreases at DTI=50.80: fr...
7,DTI,45.9,-0.462237,Effect dramatically decreases at DTI=45.90: fr...
8,DTI,41.0,-0.311028,Effect dramatically decreases at DTI=41.00: fr...
9,Fin_amt,56924.0,-0.280223,Effect dramatically decreases at Fin_amt=56924...


---
# Segment Discovery

## 5. Auto Segment Discovery

**What it does:** Finds groups of applicants where the MODEL REASONS DIFFERENTLY

**Why it's different from traditional segmentation:**
- Traditional: Groups by demographics (FICO bands, loan size tiers)
- This: Groups by SHAP patterns (how model explains predictions)

Reveals behavioral segments like:
- "FICO Driven" - predictions mainly driven by credit score
- "DTI Sensitive" - predictions driven by debt ratios
- "High LTV Risk" - predictions driven by loan-to-value

**NATIVE SHAP DOESN'T HAVE THIS!**

In [16]:
# Discover behavioral segments
segments = interp.discover_segments(n_segments=4, top_n_features=4)

# Print summary
print(interp.get_segment_summary(segments))

DISCOVERED SEGMENTS SUMMARY

Method: kmeans
Number of segments: 4
Features used for clustering: FICO, LTV, DTI, Fin_amt...

----------------------------------------------------------------------
SEGMENT 0: Fico Driven (strongly dominated by FICO)
----------------------------------------------------------------------
Size: 241 observations (12.0% of total)
Target rate: 29.0%

Top Drivers:
  FICO                           -0.6004
  LTV                            -0.2540
  Fin_amt                        -0.2384
  DTI                            -0.1491

----------------------------------------------------------------------
SEGMENT 1: Fico Driven (strongly dominated by FICO)
----------------------------------------------------------------------
Size: 936 observations (46.8% of total)
Target rate: 43.2%

Top Drivers:
  FICO                           +0.5485
  LTV                            -0.2298
  DTI                            +0.1273
  Fin_amt                        +0.0671

------------

In [17]:
# Visualize segment profiles
fig = interp.plot_segment_profiles(segments, top_n_features=4)
fig.show()

In [18]:
# Compare feature importance across segments
fig = interp.plot_segment_comparison(segments, top_n=4)
fig.show()

---
# Local Explanations

## 6. Waterfall & Force Plots

**What it shows:** How a SINGLE prediction was made

- Base value = average model output
- Each feature pushes prediction up or down
- Final prediction = sum of all contributions

Essential for:
- Explaining individual loan decisions
- Understanding edge cases
- Regulatory compliance (explainability)

In [19]:
# Find a high-probability booking and a low-probability one
# IMPORTANT: Use X_shap (the sampled data used for SHAP) for local explanations
X_shap = interp.X_shap
y_proba_shap = model.predict_proba(X_shap)[:, 1]

high_prob_idx = np.argmax(y_proba_shap)  # Highest booking probability in sampled data
low_prob_idx = np.argmin(y_proba_shap)   # Lowest booking probability in sampled data
mid_idx = np.argsort(y_proba_shap)[len(y_proba_shap)//2]  # Middle

print(f"High probability application: idx={high_prob_idx}, prob={y_proba_shap[high_prob_idx]:.3f}")
print(f"Low probability application: idx={low_prob_idx}, prob={y_proba_shap[low_prob_idx]:.3f}")
print(f"Mid probability application: idx={mid_idx}, prob={y_proba_shap[mid_idx]:.3f}")

# Show feature values for high prob case
print(f"\nHigh probability case features:")
print(X_shap.iloc[high_prob_idx])

High probability application: idx=242, prob=0.856
Low probability application: idx=544, prob=0.005
Mid probability application: idx=232, prob=0.315

High probability case features:
FICO         809.0
LTV           78.0
DTI           12.5
Fin_amt    15373.0
Name: 242, dtype: float64


In [20]:
# Waterfall plot for HIGH probability prediction
fig = interp.plot_waterfall(high_prob_idx, top_n=4)
fig.show()

print("\nThis application has HIGH booking probability because:")
print(interp.explain_observation_text(high_prob_idx, top_n=4))


This application has HIGH booking probability because:
Observation #242 Explanation
Base prediction (average): -0.754
Final prediction: 1.780
Total SHAP adjustment: +2.534

Top Contributing Features:
--------------------------------------------------
  FICO = 809.0  (+1.5067, increases prediction)
  DTI = 12.5  (+0.5063, increases prediction)
  LTV = 78.0  (+0.3648, increases prediction)
  Fin_amt = 15373.0  (+0.1561, increases prediction)


In [21]:
# Waterfall plot for LOW probability prediction
fig = interp.plot_waterfall(low_prob_idx, top_n=4)
fig.show()

print("\nThis application has LOW booking probability because:")
print(interp.explain_observation_text(low_prob_idx, top_n=4))


This application has LOW booking probability because:
Observation #544 Explanation
Base prediction (average): -0.754
Final prediction: -5.211
Total SHAP adjustment: -4.457

Top Contributing Features:
--------------------------------------------------
  FICO = 492.0  (-2.3668, decreases prediction)
  DTI = 57.0  (-1.2332, decreases prediction)
  LTV = 105.0  (-0.6993, decreases prediction)
  Fin_amt = 30855.0  (-0.1580, decreases prediction)


In [22]:
# Force plot (alternative horizontal view)
fig = interp.plot_force(high_prob_idx, top_n=4)
fig.show()

In [23]:
# Compare multiple applications side by side
fig = interp.plot_multiple_observations([high_prob_idx, mid_idx, low_prob_idx], top_n=4)
fig.show()

---
## 7. Interaction Detection

**What it shows:** Which feature pairs interact most

For credit data:
- Does FICO + DTI interact? (credit score matters differently at different debt levels)
- Does LTV + Fin_amt interact? (loan size matters differently at different LTV)

In [24]:
interactions = interp.detect_interactions(top_n=10, method='shap_variance')
print("Top Feature Interactions:")
interactions

Top Feature Interactions:


,feature_1,feature_2,interaction_strength
0,FICO,LTV,0.209973
1,FICO,DTI,0.060384
2,FICO,Fin_amt,0.027786
4,LTV,Fin_amt,0.019320
5,DTI,Fin_amt,0.011570
3,LTV,DTI,0.008683


---
## 8. Prediction Surface Visualizations

**What it shows:** Predicted booking probability across two features

**Unique to this package** - shows actual predictions (Y), not SHAP values

Business questions:
- How do FICO and DTI together affect booking?
- What combinations lead to highest approval?

In [25]:
# Heatmap: FICO × DTI
fig = interp.plot_interaction_contour('FICO', 'DTI', n_grid=30)
fig.show()

print("\nBusiness insight: How does the combination of credit score and debt ratio affect booking?")


Business insight: How does the combination of credit score and debt ratio affect booking?


In [26]:
# Heatmap: FICO × LTV
fig = interp.plot_interaction_contour('FICO', 'LTV', n_grid=30)
fig.show()

print("\nBusiness insight: How do credit score and loan-to-value interact?")


Business insight: How do credit score and loan-to-value interact?


In [27]:
# Heatmap: LTV × Fin_amt
fig = interp.plot_interaction_contour('LTV', 'Fin_amt', n_grid=30)
fig.show()

print("\nBusiness insight: How do loan-to-value and financing amount interact?")


Business insight: How do loan-to-value and financing amount interact?


In [28]:
# 3D surface plot: FICO × DTI
fig = interp.plot_interaction_surface_3d('FICO', 'DTI', n_grid=25)
fig.show()

In [29]:
# 3D surface plot: FICO × LTV
fig = interp.plot_interaction_surface_3d('FICO', 'LTV', n_grid=25)
fig.show()

---
## 9. Model Performance

**Includes:** Confusion matrix, ROC curve, metrics summary

In [30]:
performance = interp.plot_performance()

performance['metrics_summary'].show()

In [31]:
performance['confusion_matrix'].show()

In [32]:
performance['roc_curve'].show()

---
# Summary: Credit Model Interpretation

## Key Insights from This Analysis

### Feature Importance
- Identified which credit factors (FICO, DTI, LTV, Fin_amt) drive booking decisions
- Quantified the relative importance of each factor

### Threshold Discovery
- Found specific FICO score breakpoints where approval probability changes
- Identified DTI and LTV cutoffs that matter

### Segment Discovery
- Found groups of applicants where the model reasons differently
- Revealed hidden patterns in how credit factors combine

### Local Explanations
- Can explain individual loan decisions
- Useful for regulatory compliance and customer communication

### Interaction Effects
- Visualized how feature combinations affect booking probability
- Identified non-linear relationships

## Why This Package Beats Native SHAP

- **Interactive Plotly** vs static matplotlib
- **Business-focused** - shows predictions, not just SHAP
- **Discovers insights** - thresholds, segments, interactions
- **Simple API** - one Interpreter class
- **Great for presentations** - interactive, professional visuals